In [1]:
# Assumption regarding input and output data
# expecting to get numerous csv-files rom different sources with user_id as a merging key
# encoding is expected to be cp1251
# desired df structure (one merged file)
# user_id    item_id    signals(implicit data)    date    meta_data => (age, gender)   ....
# some sorts of analysis and data wranglings are expected to be performed in Jupyter Notebook
# prepare data for further preprocessing and customization for a particular algorithm

In [2]:
# Importing all needed stuff

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
plt.rcParams['figure.figsize'] = (12,8)

In [4]:
# Import special ad-hoc libs here

In [5]:
from datetime import datetime
from collections import defaultdict

In [6]:
PATH = r'data/'

In [7]:
readers_file = 'читатели.xlsx'

In [8]:
issue1_file = 'Выдача_1.xlsx'
issue2_file = 'Выдача_2.xlsx'

In [29]:
samples1_file = 'Экземпляры.xlsx'
samples2_file = 'Экземпляры_2.xlsx'

In [30]:
catalogue_file = 'Каталог.xlsx'

#### Read & Explore data

##### Readers

In [10]:
df_readers = pd.read_excel(os.path.join(PATH, readers_file))

In [12]:
df_readers.head()

,ID читателя,Дата рождения
0,366670,1974-10-14 00:00:00
1,235264,1978-06-01 00:00:00
2,133225,1976-01-06 00:00:00
3,556716,1989-01-28 00:00:00
4,52086,1955-08-14 00:00:00


In [14]:
df_readers.shape

(562133, 2)

In [17]:
df_readers.describe(include='O')

,Дата рождения
count,562133
unique,32872
top,2018-12-12 00:00:00
freq,431


In [18]:
df_readers.isnull().sum()

ID читателя      0
Дата рождения    0
dtype: int64

In [25]:
df_readers.dtypes

ID читателя       int64
Дата рождения    object
dtype: object

In [31]:
df_readers.iloc[100,1]

Timestamp('1999-06-09 00:00:00')

In [32]:
def extract_year(x):
    if isinstance(x,str):
        return x.split('-')[0]
    else:
        return x.strftime('%Y-%m-%d-%H.%M.%S').split('-')[0]

In [39]:
df_readers['Year'] = df_readers['Дата рождения'].apply(lambda x: extract_year(x))

In [40]:
cur_year = 2020
def age(x):
    return int(cur_year - int(x))

In [41]:
df_readers['Age'] = df_readers['Year'].apply(lambda x: age(x))

In [42]:
df_readers.head()

,ID читателя,Дата рождения,Year,Age
0,366670,1974-10-14 00:00:00,1974,46
1,235264,1978-06-01 00:00:00,1978,42
2,133225,1976-01-06 00:00:00,1976,44
3,556716,1989-01-28 00:00:00,1989,31
4,52086,1955-08-14 00:00:00,1955,65


In [33]:
def extract_month(x):
    if isinstance(x,str):
        return x.split('-')[1]
    else:
        return x.strftime('%Y-%m-%d-%H.%M.%S').split('-')[1]

In [44]:
df_readers['Month'] = df_readers['Дата рождения'].apply(lambda x: extract_month(x))

In [46]:
df_readers.drop(columns=['Дата рождения', 'Year'], inplace=True)

In [47]:
df_readers.head()

,ID читателя,Age,Month
0,366670,46,10
1,235264,42,06
2,133225,44,01
3,556716,31,01
4,52086,65,08


In [48]:
df_readers.to_csv('data_clean/readers.csv')

##### Issues 1 & 2

In [9]:
%%time
df1 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача_1')

Wall time: 7min 22s


In [10]:
df1.head()

,ИД выдачи,ИД читателя,Инвентарный номер,Штрих-код,Дата выдачи,Дата сдачи (предполагаемая),Состояние,Unnamed: 7,Состояния,Unnamed: 9
0,1,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,NaN,6544.0,На руках
1,2,179,03:0000005761,381000196420,2018-04-13 15:00:00,2018-04-16 15:00:00,6548,NaN,6545.0,Переоформлен
2,3,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,NaN,6548.0,Размещен в фонде
3,4,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,NaN,NaN,NaN
4,5,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6548,NaN,NaN,NaN


In [11]:
df1.drop(columns=['Unnamed: 7', 'Состояния', 'Unnamed: 9'], inplace=True)

In [13]:
df2 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача_2', usecols=range(0, 7))

In [14]:
df2.head()

,ИД выдачи,ИД читателя,Инвентарный номер,Штрих-код,Дата выдачи,Дата сдачи (предполагаемая),Состояние
0,1,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545
1,2,179,03:0000005761,381000196420,2018-04-13 15:00:00,2018-04-16 15:00:00,6548
2,3,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545
3,4,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545
4,5,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6548


In [15]:
df3 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача_3', usecols=range(0, 7))

In [16]:
df4 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача_4', usecols=range(0, 7))

In [17]:
df5 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача_5', usecols=range(0, 7))

In [19]:
df6 = pd.read_excel(os.path.join(PATH, issue1_file), sheet_name='Выдача 6', usecols=range(0, 7))

In [ ]:
#############################################################################

In [20]:
df7 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_7', usecols=range(0, 7))

In [21]:
df8 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_8', usecols=range(0, 7))

In [22]:
df9 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_9', usecols=range(0, 7))

In [23]:
df10 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_10', usecols=range(0, 7))

In [24]:
df11 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_11', usecols=range(0, 7))

In [25]:
df12 = pd.read_excel(os.path.join(PATH, issue2_file), sheet_name='Выдача_12', usecols=range(0, 7))

In [26]:
df_issues = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis=0)

In [39]:
df_issues.tail()

,ИД выдачи,ИД читателя,Инвентарный номер,Штрих-код,Дата выдачи,Дата сдачи (предполагаемая),Состояние,Year_issue,Month_issue
423355,11423388,566211,09:0000922563,980012545389,2020-10-30 15:00:00,2020-11-29 15:00:00,6544,2020,10
423356,11423389,110298,08:0000227792,880002994725,2020-10-10 15:00:00,2020-11-29 15:00:00,6544,2020,10
423357,11423390,110298,Б08:00001928,880000269153,2020-10-01 15:00:00,2020-11-29 15:00:00,6544,2020,10
423358,11423391,110298,Б08:00027425,880007222816,2020-10-01 15:00:00,2020-11-29 15:00:00,6544,2020,10
423359,11423392,535365,08:0001063365,880008412483,2020-10-30 15:00:00,2020-11-29 15:00:00,6544,2020,10


In [28]:
df_issues.shape

(11420807, 7)

In [34]:
df_issues['Year_issue'] = df_issues['Дата выдачи'].apply(lambda x: extract_year(x))

In [35]:
df_issues['Month_issue'] = df_issues['Дата выдачи'].apply(lambda x: extract_month(x))

In [45]:
def dur_issue(row):
    date_in = row['Дата выдачи']
    date_out = row['Дата сдачи (предполагаемая)']
    if isinstance(date_in,str):
        date_in = datetime.strptime(date_in, '%Y-%m-%d %H:%M:%S')
    if isinstance(date_out,str):
        date_out = datetime.strptime(date_out, '%Y-%m-%d %H:%M:%S')
    return date_out - date_in

In [47]:
df_issues['Duration_issue'] = df_issues.apply(lambda row: dur_issue(row), axis=1)

In [48]:
df_issues.head()

,ИД выдачи,ИД читателя,Инвентарный номер,Штрих-код,Дата выдачи,Дата сдачи (предполагаемая),Состояние,Year_issue,Month_issue,Duration_issue
0,1,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,2018,04,0 days
1,2,179,03:0000005761,381000196420,2018-04-13 15:00:00,2018-04-16 15:00:00,6548,2018,04,3 days
2,3,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,2018,04,0 days
3,4,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6545,2018,04,0 days
4,5,179,03:0000005767,381000177719,2018-04-13 15:00:00,2018-04-13 15:00:00,6548,2018,04,0 days


In [49]:
df_issues.drop(columns=['Дата выдачи', 'Дата сдачи (предполагаемая)', 'Штрих-код'], inplace=True)

In [50]:
map_ = {
    '6544':'На руках',
    '6545':'Переоформлен',
    '6548':'Размещен в фонде',
}

In [51]:
df_issues['Состояние'] = df_issues['Состояние'].map(map_)

In [52]:
df_issues.to_csv('data_clean/books_issues.csv')

##### Samples

In [53]:
%%time
df1 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_1', usecols=(1,2))

Wall time: 7min 28s


In [54]:
df2 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_2', usecols=(1,2))

In [55]:
df3 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_3', usecols=(1,2))

In [56]:
df4 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_4', usecols=(1,2))

In [57]:
df5 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_5', usecols=(1,2))

In [58]:
df6 = pd.read_excel(os.path.join(PATH, samples1_file), sheet_name='Фонд_6', usecols=(1,2))

In [59]:
df7 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_7', usecols=(1,2))

In [60]:
df8 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_8', usecols=(1,2))

In [61]:
df9 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_9', usecols=(1,2))

In [62]:
df10 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_10', usecols=(1,2))

In [63]:
df11 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_11', usecols=(1,2))

In [64]:
df12 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_12', usecols=(1,2))

In [65]:
df13 = pd.read_excel(os.path.join(PATH, samples2_file), sheet_name='Фонд_13', usecols=(1,2))

In [66]:
df_samples = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], axis=0)

In [67]:
df_samples.head()

,ИД Каталожной записи,Инвентарный номер
0,1,09:0000120386
1,1,09:0000152427
2,1,09:0000240622
3,1,09:0000278133
4,1,05:0000257728


In [68]:
df_samples.shape

(12728880, 2)

In [69]:
df_samples.to_csv('data_clean/books_samples.csv')

##### Catalogue

In [70]:
df_cat1 = pd.read_excel(os.path.join(PATH, catalogue_file), sheet_name='Каталог_1')

In [71]:
df_cat2 = pd.read_excel(os.path.join(PATH, catalogue_file), sheet_name='Каталог_2')

In [72]:
df_cat = pd.concat([df_cat1, df_cat2], axis=0)

In [73]:
df_cat.shape

(1370881, 10)

In [74]:
df_cat.head()

,doc_id,p100a,p245a,p260a,p260b,p260c,p490a,p650a,p084a,p521a
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+
1,2,NaN,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,NaN,Наука. Науковедение,72,12+
2,3,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
3,4,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
4,5,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+


In [75]:
df_cat.columns = [
    'ID книги', 'Автор', 'Название', 'Город (место издания)','Издательство', 'Год издания', 'Серия', 'Жанр' , 'Полка', 'Возратные ограничения'
]

In [76]:
df_cat.head()

,ID книги,Автор,Название,Город (место издания),Издательство,Год издания,Серия,Жанр,Полка,Возратные ограничения
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,Татьяна Устинова. Первая среди лучших,Художественная литература ; Российский детектив,84(2Рос)6,16+
1,2,NaN,Наука и жизнь,Москва,АНО Редакция журнала «Наука и жизнь»,1938-,NaN,Наука. Науковедение,72,12+
2,3,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
3,4,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+
4,5,NaN,Наука и жизнь,NaN,NaN,2018,NaN,Наука. Науковедение,72,12+


In [77]:
df_cat.drop(columns=['Город (место издания)', 'Издательство', 'Полка'], inplace=True)

In [78]:
df_cat['Жанр'].value_counts()

Художественная литература                                                                                                                                                                     489182
История России                                                                                                                                                                                 59370
Литература для детей и юношества                                                                                                                                                               22982
История и критика литературы России                                                                                                                                                            22501
История в целом. Всемирная история                                                                                                                                                             22104
               

In [80]:
len(df_cat['Жанр'].unique())

9073

In [81]:
df_cat['Жанр'].isnull().sum()

80525

In [82]:
df_cat.to_csv('data_clean/books_catalogue.csv')

#### Sections

In [83]:
relations = 'MegaRelation_hackaton.csv'
pupil = 'Pupil_hackaton.csv'
request = 'request_hackaton.csv'
service = 'services_hackaton.csv'

In [88]:
df_relations = pd.read_csv(os.path.join(PATH, relations), sep=';')

In [89]:
df_relations.head()

,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
0,1325954,3.0,25969.0,08.06.2015 14:47:19,30,950832.0,39178,08.06.2015 0:00:00,29.02.2016 19:28:12,NaN,NaN,3558114.0,01.09.2014 0:00:00,01.09.2021 0:00:00
1,2879918,3.0,158129.0,29.10.2015 18:35:21,30,989658.0,39178,01.09.2010 0:00:00,29.02.2016 19:23:59,NaN,NaN,3557885.0,01.09.2014 0:00:00,01.09.2021 0:00:00
2,2901258,3.0,304476.0,02.11.2015 14:05:56,30,950764.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558103.0,01.09.2014 0:00:00,01.09.2021 0:00:00
3,2912724,3.0,1139915.0,03.11.2015 14:29:27,30,950786.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558105.0,01.09.2014 0:00:00,01.09.2021 0:00:00
4,2912819,3.0,298767.0,03.11.2015 14:34:15,30,950799.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558106.0,01.09.2014 0:00:00,01.09.2021 0:00:00


In [90]:
df_relations.drop(columns=['id_организации', 'дата_отчисления', 'предыдущая_запись_зачисления',
                          'следующая_запись_зачисления', 'Плановая_дата_начала_занятий',
                          'Плановая_дата_окончания_занятий'], inplace=True)

In [91]:
df_relations.to_csv('data_clean/sections_relations.csv')

In [92]:
df_services = pd.read_csv(os.path.join(PATH, service), sep=';')

In [93]:
df_services.head()

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 12:50:32,"7,0000",лет
1,39181,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 13:04:41,"7,0000",лет
2,212072,бесплатно,3000001,30,Индивидуальное,Архитектура,27.10.2015 15:59:48,"7,0000",лет
3,212517,бесплатно,3000001,30,Индивидуальное,Архитектура,29.10.2015 18:45:49,"7,0000",лет
4,39913,бесплатно,3000003,45,Индивидуальное,Декоративно-прикладное искусство,17.07.2013 20:35:28,"7,0000",лет


In [96]:
df_services.drop(columns=['Классификатор_услуги', 'id_организации'], inplace=True)

In [97]:
df_services.to_csv('data_clean/sections_services.csv')

#### Activities

In [99]:
df_activities = pd.read_excel(os.path.join(PATH, 'Мероприятия.xlsx'), usecols=(1,7,8,10,18,23,24))

In [100]:
df_activities.head()

,Название мероприятия,Тип мероприятия,Направленность мероприятия,Наименование праздника или события,Округ,Возрастная категория,Целевая аудитория
0,Фестивлаь современных молодежных уличных культ...,фестиваль,культурно-досуговое,NaN,NaN,от 20 до 40,NaN
1,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...","танцевальный вечер, бал",патриотическое,9 мая. День Победы,Южный административный округ,от 40 до 85,"работающая молодежь, взрослые, пенсионеры"
2,Врасплох,выставка,антитеррор,NaN,Южный административный округ,от 16 до 100,"школьники, работающая молодежь, студенческая м..."
3,Праздник Новогодней елки. Новогодний концерт в...,"концерт, музыкальное представление",культурно-досуговое,1 января. Новый год,NaN,от 5 до 14,"дети дошкольного возраста, школьники"
4,"Выставка к Рождеству ""Сувенир Года""",выставка,культурно-досуговое,1 января. Новый год,Южный административный округ,от 15 до 80,"дети дошкольного возраста, школьники, работающ..."


In [ ]:
df_activities.to_csv('data_clean/activities_services.csv')

#### Check & Validate & Wrangle data

1. Check relevant columns
2. Check na, fullfill or remove NA
3. Check type and format
4. Build up initial hypothesises regarding data (groupby/pivot for better understanding)
5. Check for duplicates
6. Check for outliers
7. Reingeneer features on the spot if applicable
8. Remove trash
9. Define merging keys

#### Merging data

In [ ]:
# df = df.merge(df1, how='left', left_on='', right_on='', suffixes=('_x', '_y'), indicator=True)

#### Feature Engineering

1. Define which features may be relevant
2. Define dtypes
3. Which ones can be turned into dummies
4. Which ones can be removed
5. Which ones can be turned into categorical (bins)
6. Which ones needed to be newly created/modified
7. Is clustering or feature selection required?
8. ...

#### Final data look-up

In [ ]:
# df_merged.to_csv('../data_algo/df_merged.csv')